## Calculate the single particle, electron/hole spin matrix elements, including off-diagonal terms.
The single single particle matrix elements are then used to calculate the excitonic matrix elements.

Steps:
1. Do a Quantum Espresso wavefunction calculation. The same wavefunction exactly should be used as the fine grid in absorption.
2. Convert the wavefunctions to BerkeleyGW format using pw2bgw.x
3. Convert binary wavefunction file to hdf5 format using: 


In [ ]:
/BGW_dir/bin/wfn2hdf.x BIN WFN WFN.h5

4. Run calculate_offdiagonal_spin.py after changing the four input arguments:

    nv: number of valence bands you wish to include<br>
    nc: number of conduction bands you wish to include<br>
    wfn_path : path to directory where the WFN.h5 file is stored<br>
    spin_direction: 1 to calculate the spin matrix elements along the X direction, 2 for the Y direction and 3 for the Z direction.<br>

This script multiplies two electron/hole DFT wavefunctions of bands $m$ and $n$, with momentum $k$, with the Pauli matrice of direction $i$:

$\sigma_{mnk}^i = \left<mk\left|\hat{\sigma}^i\right|nk\right>$


In [ ]:
python calculate_offdiagonal_spin.py nv nc wfn_path spin_direction

Now you have the spin matrix elements including off-diagonal elements between different bands stored at the directory ‘off_diag_spin_output’ which was created in the directory of WFN.h5.
You get one pickle file for each direction which contains complex numbers in a numpy array with the dimensions of [nk, nv+nc, nv+nc].

These elements can be used to calculate the exciton spin matrix elements including off-diagonal elements which are especially interesting for degenerate excitons.

## Calculate exciton spin matrix elements from DFT and GW-BSE including off-diagonal elements

1. Create a directory with links to the calculated single-particle spin elements, eigenvectors.h5 from BerekeleyGW and wfn.in which was used. Notice that the WFN.h5 file (or WFN in binary format) should be the exact same one you used as the fine grid in the absorption calculation. The example below is for spin in the Z direction but you can change it to X or Y.

In [ ]:
mkdir exciton_spin
cd exciton_spin
ln -s /path1/eigenvectors.h5 .
ln -s /path2/Sz .
ln -s /path2/wfn.in .

2. Run the script that multiplies the electron/hole wavefunction with the exciton wavefunctions and gives a nS x nS matrix of exciton spin matrix elements, where nS is the number of lowest energy excitons chosen as an input parameter, at one of the three directions (1=x, 2=y, 3=z). You should use '.' for the path if you are in the directorn where the single particle spins and the excitonic eigenvectors are stored or linked at.

This script multiplies the previously calculated single-particle spin matrix elements along direction $i$ ($\sigma^i_{mnk})$, with the excitonic wavefunctions of states $S,S'$ calculated previously by solving the Bethe-Salpeter equation and spanned in the electron-hole basis set with $A$ the spanning coefficients:

$
\Sigma_{SS'}^i=\left<S\left|\hat{\Sigma}^i\right|S'\right>=\sum_{vck}{\left(\sum_{c'}{A^{S*}_{vck}A^{S'}_{vc'k}\sigma^i_{cc'k}}-\sum_{v'}{A^{S*}_{vck}A^{S'}_{v'ck}\sigma^i_{vv'k}}\right)}
$

In [ ]:
python calculate_offdiagonal_exciton_spin path spin_direction nS

Now you can observe the different diagonal and offdiagonal elements of the exciton spin, maybe diagonalize this matrix or blocks from it and do whatever you wish to.